In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from jupyterthemes import jtplot
import seaborn as sns

%matplotlib inline


In [2]:
jtplot.style(theme='monokai')

# Import data
Remember! I'm only using the working lyrics on day one as I am still importing lyric data

In [16]:
chart_df = pd.read_csv('../data/all_charts.csv')
lyrics_df = pd.read_csv('../data/working_lyrics.csv')
spotify_df = pd.read_csv('../data/spotify_data.csv')

/Users/AndrewKraemer/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (18,19,20,21,22,23,24,25,26,27,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [17]:
chart_data.head(2)

,chartDate,title,artist,peakPos,lastPos,weeks,rank,change,spotifyID
0,1958-08-16,Poor Little Fool,Ricky Nelson,1,1,11,1,0,33FPsMEl3UwpytDuyf9VYq
1,1958-08-16,Nel Blu Dipinto Di Blu (VolarÃÂ©),Domenico Modugno,1,54,16,2,+52,NaN


In [18]:
lyrics_df.head(2)

,Song,Artist,spotifyID,Lyrics,Source
0,poor little fool,ricky nelson,33FPsMEl3UwpytDuyf9VYq,i used to play around with hearts that hastene...,1.0
1,nel blu dipinto di blu volar~a^a,domenico modugno,NaN,not set yet,NaN


In [19]:
spotify_df.head(2)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_uri,duration_ms,time_signature,key_mode
0,0.549,0.336,C,-11.669,major,0.0290,0.6520,0.000,0.134,0.802,154.958,33FPsMEl3UwpytDuyf9VYq,153933.0,4,C major
1,0.696,0.743,C#,-6.623,major,0.0439,0.0836,0.197,0.119,0.776,137.276,0nG4SbrAONIFLIwQqegooj,138747.0,4,C# major


In [20]:
# keep relevant columns
# chart_data.columns.tolist()

chart_cols = ['chartDate', 'title', 'artist', 'peakPos', 'lastPos', 'weeks', 'rank', 'change','spotifyID']
chart_data = chart_df[chart_cols]

In [46]:
df = pd.merge(chart_data, spotify_df, left_on='spotifyID', right_on = 'track_uri', how='inner')

In [47]:
chart_len = chart_data.shape[0]
spot_len = df.shape[0]
print('chart_songs: ' + str(chart_len), 
     '\n spotify_songs:' + str(spot_len),
     '\n percent_overlap: ' + str(round(spot_len / chart_len, 4)*100) + '%')

chart_songs: 304643 
 spotify_songs:258830 
 percent_overlap: 84.96000000000001%


In [35]:
print('hello world')

hello world
